In [ ]:
!pip install transformers
!pip install torch
!pip install nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
!pip install PyMuPDF
!pip install sumy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 98.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.4 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=5e08df319a3ab4b30e2c665d7d57417f0270bec40dbe97491fb1e68d160e080a
  Stored in directory: /root/.cache/pip/wheels/4d/57/58/7e3d7fedf51fe248b7fcee3df6945ae28638e22cddf01eb92b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=55e98719777048aa8dbdf3a44495b8a04df6791f3dde9c7f251ac500ce0fbfd8
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built breadability docopt


In [24]:
# Transformers
from transformers import pipeline

# NLTK for sentence tokenization
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Add this line to download the required resource

# File handling
import fitz  # PyMuPDF for PDF
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Other
import os


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Hugging Face BART summarizer
bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


Device set to use cuda:0


In [ ]:
def summarize_with_bart(text, max_length=130, min_length=30):
    summary = bart_summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    return summary[0]['summary_text']


In [ ]:
def summarize_with_sumy(text, sentence_count=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return " ".join(str(sentence) for sentence in summary)


In [ ]:
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


In [ ]:
def read_pdf_file(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text


In [26]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Detect file type
if filename.endswith('.txt'):
    text_data = read_txt_file(filename)
elif filename.endswith('.pdf'):
    text_data = read_pdf_file(filename)
else:
    raise ValueError("Unsupported file type. Upload only .txt or .pdf files.")


Saving The tiger (Panthera tigris) is a la.txt to The tiger (Panthera tigris) is a la (1).txt


In [27]:
print("Choose summarization model:")
print("1. BART (transformers - abstractive)")
print("2. LSA (sumy - extractive)")

choice = input("Enter 1 or 2: ")

if choice == "1":
    summary = summarize_with_bart(text_data)
elif choice == "2":
    summary = summarize_with_sumy(text_data)
else:
    summary = "Invalid choice."


Choose summarization model:
1. BART (transformers - abstractive)
2. LSA (sumy - extractive)
Enter 1 or 2: 1


In [28]:
print("\n--- ORIGINAL TEXT ---\n")
print(text_data[:1000])  # print first 1000 chars

print("\n--- SUMMARY ---\n")
print(summary)



--- ORIGINAL TEXT ---

Mobil Oil Corporation, now known as just Mobil, is a petroleum brand owned and operated by American oil and gas corporation ExxonMobil, formerly known as Exxon, which took its current name after it and Mobil merged in 1999.

A direct descendant of Standard Oil, Mobil was originally known as the Standard Oil Company of New York (shortened to Socony) after Standard Oil was split into 43 different entities in a 1911 Supreme Court decision. Socony merged with Vacuum Oil Company, from which the Mobil name first originated, in 1931 and subsequently renamed itself to "Socony-Vacuum Oil Company".[2] Over time, Mobil became the company's primary identity, which prompted a renaming in 1955 to the "Socony Mobil Oil Company", and then in 1966 to the "Mobil Oil Corporation". Mobil credits itself with being the first company to introduce paying at the pump at its gas stations, the first company to produce jet aviation fuel, as well as the first company to introduce a mobile p